Exercise Predictive monitoring - Part 2
Filtering prefixes, aggregation and index-based encoding. Repeat the following tasks for the 2 event logs: small (synthetic) and BPIC_2017 (real). Commit the solutions on your github repository.






In [1]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime 

%matplotlib inline

# _1. Import an event log as a pandas dataframe


In [2]:
BPIC17 = pd.read_csv('Sampled_BPIC17_offer_log.csv', index_col = ['index'])

In [3]:
BPIC17.rename_axis(None, inplace=True)
BPIC17.head(2)

,Case ID,Activity,Resource,Complete Timestamp,Variant,Variant index,(case) Accepted,(case) ApplicationID,(case) CreditScore,(case) FirstWithdrawalAmount,(case) MonthlyCost,(case) NumberOfTerms,(case) OfferedAmount,(case) Selected,Action,EventID,EventOrigin,OfferID,lifecycle:transition
0,Offer_247135719,O_Create Offer,User_17,2016/01/02 18:17:05.720,Variant 8,8,True,Application_196483749,0,10000.0,201.76,57,10000.0,False,Created,Offer_247135719,Offer,NaN,complete
1,Offer_247135719,O_Created,User_17,2016/01/02 18:17:08.762,Variant 8,8,True,Application_196483749,0,10000.0,201.76,57,10000.0,False,statechange,OfferState_124849367,Offer,Offer_247135719,complete


In [4]:
BPIC17.head(2)

,Case ID,Activity,Resource,Complete Timestamp,Variant,Variant index,(case) Accepted,(case) ApplicationID,(case) CreditScore,(case) FirstWithdrawalAmount,(case) MonthlyCost,(case) NumberOfTerms,(case) OfferedAmount,(case) Selected,Action,EventID,EventOrigin,OfferID,lifecycle:transition
0,Offer_247135719,O_Create Offer,User_17,2016/01/02 18:17:05.720,Variant 8,8,True,Application_196483749,0,10000.0,201.76,57,10000.0,False,Created,Offer_247135719,Offer,NaN,complete
1,Offer_247135719,O_Created,User_17,2016/01/02 18:17:08.762,Variant 8,8,True,Application_196483749,0,10000.0,201.76,57,10000.0,False,statechange,OfferState_124849367,Offer,Offer_247135719,complete


In [5]:
BPIC17.columns

Index(['Case ID', 'Activity', 'Resource', 'Complete Timestamp', 'Variant',
       'Variant index', '(case) Accepted', '(case) ApplicationID',
       '(case) CreditScore', '(case) FirstWithdrawalAmount',
       '(case) MonthlyCost', '(case) NumberOfTerms', '(case) OfferedAmount',
       '(case) Selected', 'Action', 'EventID', 'EventOrigin', 'OfferID',
       'lifecycle:transition'],
      dtype='object')

# _2.Create an event log that retains only the attributes caseid, activity, timestamp, resource and outcome. Rename the attributes as “caseid, activity, ts, resource, outcome” if names are different (note: we should indicate which one is the outcome label)


In [6]:
# resource_anomaly_type : outcome
df_original = BPIC17[['Case ID', 'Activity', 'Complete Timestamp', 'Resource', '(case) Accepted']]
df_original.head()

,Case ID,Activity,Complete Timestamp,Resource,(case) Accepted
0,Offer_247135719,O_Create Offer,2016/01/02 18:17:05.720,User_17,True
1,Offer_247135719,O_Created,2016/01/02 18:17:08.762,User_17,True
2,Offer_247135719,O_Sent (online only),2016/01/02 18:19:21.330,User_17,True
3,Offer_247135719,O_Cancelled,2016/01/02 18:21:26.034,User_17,True
4,Offer_941964966,O_Create Offer,2016/01/02 18:21:42.022,User_17,True


In [7]:
cols = ['caseid', 'activity', 'ts', 'resource', 'outcome']
df_original.columns = cols
df_original.head()

,caseid,activity,ts,resource,outcome
0,Offer_247135719,O_Create Offer,2016/01/02 18:17:05.720,User_17,True
1,Offer_247135719,O_Created,2016/01/02 18:17:08.762,User_17,True
2,Offer_247135719,O_Sent (online only),2016/01/02 18:19:21.330,User_17,True
3,Offer_247135719,O_Cancelled,2016/01/02 18:21:26.034,User_17,True
4,Offer_941964966,O_Create Offer,2016/01/02 18:21:42.022,User_17,True


# _3.Create a function that does aggregation encoding of an event log:
Activity and resource are aggregated using frequency
Timestamp are aggregated using average


In [8]:
def aggregation(df):
  # create new df for result
  df_result = pd.DataFrame( columns= [ 'List_Activity', 'List_Resource'])

  # List of Activity and Resource
  df_result.loc[:,'List_Activity']= df.groupby('caseid')['activity'].apply(list)
  df_result.loc[:,'List_Resource']= df.groupby('caseid')['resource'].apply(list)

  # Average of timestamps
  df.loc[:,'ts'] = pd.to_datetime(df.loc[:,'ts'])
  tsMin = df.groupby('caseid')['ts'].min()
  df = pd.merge(df, tsMin.reset_index(), on= 'caseid', how='outer', suffixes = ('','_min'))
  df.loc[:,'ts_Gap'] = df['ts'] - df['ts_min']
  df_result.loc[:,'Avg_ts']= df.groupby('caseid')['ts_Gap'].apply(np.sum)/(df.groupby('caseid')['ts_Gap'].apply(len)-1)
  df_result.loc[:,'Avg_ts'] = df_result.loc[:,'Avg_ts'].dt.round('1s')

  return df_result


In [9]:
a = aggregation(df_original)
a.head()

C:\anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,List_Activity,List_Resource,Avg_ts
caseid,,,
Offer_1000681710,"[O_Create Offer, O_Created, O_Sent (mail and o...","[User_20, User_20, User_20, User_117, User_115]",3 days 14:40:42
Offer_1001553250,"[O_Create Offer, O_Created, O_Sent (mail and o...","[User_2, User_2, User_2, User_113, User_30]",4 days 07:28:50
Offer_1002136393,"[O_Create Offer, O_Created, O_Sent (mail and o...","[User_85, User_85, User_85, User_117, User_118]",10 days 11:43:53
Offer_1002236598,"[O_Create Offer, O_Created, O_Sent (mail and o...","[User_49, User_49, User_49, User_113, User_102]",2 days 07:12:34
Offer_1002530118,"[O_Create Offer, O_Created, O_Sent (mail and o...","[User_15, User_15, User_15, User_1]",10 days 04:31:56


# -4. Create a function to filter the prefixes of length L for the event log created at step 2. This function retains only the first L events of all cases that have at least L events. 


In [10]:
def prefix_df(df, n = 2):

  # the min of the number of activities is 7
  result_df = df.sort_values(by = ['caseid','ts'], ascending=True).groupby('caseid').head(n)
  result_df.reset_index(inplace = True, drop = True)

  return result_df

In [11]:
prefix_2 = prefix_df(df_original)
prefix_2.head()

,caseid,activity,ts,resource,outcome
0,Offer_1000681710,O_Create Offer,2016-02-02 18:24:54.170,User_20,True
1,Offer_1000681710,O_Created,2016-02-02 18:24:55.833,User_20,True
2,Offer_1001553250,O_Create Offer,2016-01-16 03:16:41.868,User_2,False
3,Offer_1001553250,O_Created,2016-01-16 03:16:43.056,User_2,False
4,Offer_1002136393,O_Create Offer,2016-01-29 17:46:49.529,User_85,True


In [12]:
prefix_5 = prefix_df(df_original,5)
prefix_5.head(10)

,caseid,activity,ts,resource,outcome
0,Offer_1000681710,O_Create Offer,2016-02-02 18:24:54.170,User_20,True
1,Offer_1000681710,O_Created,2016-02-02 18:24:55.833,User_20,True
2,Offer_1000681710,O_Sent (mail and online),2016-02-02 18:25:14.889,User_20,True
3,Offer_1000681710,O_Returned,2016-02-09 22:36:31.755,User_117,True
4,Offer_1000681710,O_Accepted,2016-02-10 00:55:41.368,User_115,True
5,Offer_1001553250,O_Create Offer,2016-01-16 03:16:41.868,User_2,False
6,Offer_1001553250,O_Created,2016-01-16 03:16:43.056,User_2,False
7,Offer_1001553250,O_Sent (mail and online),2016-01-16 03:16:58.528,User_2,False
8,Offer_1001553250,O_Returned,2016-01-22 17:14:16.014,User_113,False
9,Offer_1001553250,O_Accepted,2016-01-26 19:14:10.052,User_30,False


# _5. Create a function that does index-based encoding of a prefix-filtered event log.
Activity and resource are one-hot encoded
Timestamp is left as-is
*이탤릭체 텍스트*

In [13]:
def encorder(df):
  df_result = df[['outcome', 'caseid','ts']]
  df_result = df_result.merge(pd.get_dummies(df["activity"]), left_index=True, right_index=True)
  df_result = df_result.merge(pd.get_dummies(df["resource"]), left_index=True, right_index=True)

  return df_result

In [14]:
encorder(prefix_2).columns

Index(['outcome', 'caseid', 'ts', 'O_Create Offer', 'O_Created', 'User_10',
       'User_100', 'User_101', 'User_102', 'User_108', 'User_109', 'User_11',
       'User_110', 'User_112', 'User_117', 'User_119', 'User_12', 'User_13',
       'User_14', 'User_141', 'User_15', 'User_16', 'User_17', 'User_18',
       'User_19', 'User_2', 'User_20', 'User_21', 'User_22', 'User_23',
       'User_24', 'User_25', 'User_26', 'User_27', 'User_28', 'User_29',
       'User_3', 'User_30', 'User_31', 'User_33', 'User_34', 'User_35',
       'User_36', 'User_37', 'User_38', 'User_4', 'User_41', 'User_43',
       'User_44', 'User_47', 'User_49', 'User_5', 'User_50', 'User_52',
       'User_6', 'User_7', 'User_70', 'User_71', 'User_73', 'User_74',
       'User_75', 'User_76', 'User_8', 'User_85', 'User_86', 'User_87',
       'User_89', 'User_9', 'User_92', 'User_95', 'User_99'],
      dtype='object')

In [15]:
encoded_pf_2 = encorder(prefix_2)
encoded_pf_2.head()

,outcome,caseid,ts,O_Create Offer,O_Created,User_10,User_100,User_101,User_102,User_108,...,User_76,User_8,User_85,User_86,User_87,User_89,User_9,User_92,User_95,User_99
0,True,Offer_1000681710,2016-02-02 18:24:54.170,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,True,Offer_1000681710,2016-02-02 18:24:55.833,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,False,Offer_1001553250,2016-01-16 03:16:41.868,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,False,Offer_1001553250,2016-01-16 03:16:43.056,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,True,Offer_1002136393,2016-01-29 17:46:49.529,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [16]:
encoded_pf_5 = encorder(prefix_5)
encoded_pf_5.head()

,outcome,caseid,ts,O_Accepted,O_Cancelled,O_Create Offer,O_Created,O_Refused,O_Returned,O_Sent (mail and online),...,User_8,User_85,User_86,User_87,User_89,User_9,User_90,User_92,User_95,User_99
0,True,Offer_1000681710,2016-02-02 18:24:54.170,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,True,Offer_1000681710,2016-02-02 18:24:55.833,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,True,Offer_1000681710,2016-02-02 18:25:14.889,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,True,Offer_1000681710,2016-02-09 22:36:31.755,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,True,Offer_1000681710,2016-02-10 00:55:41.368,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# _6. Train a decision tree and a randomforest from the scikit-learn package for outcome label prediction using the event log created at step 3 and and step 6 for prefix length L=2 and L=5.  Split 70-30 for training-testing.

Hyperparameters
DT: max_depth=5
RF: n_estimators=100, oob_score=True


(Provide solutions for 2 event logs: artificial and real)

## Decision Tree

In [17]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import  accuracy_score         # between pred_result and real outcome
from sklearn.tree import export_graphviz            # the creating of model
import graphviz                                     # the visualization of model


### prefix_Length 2


In [18]:
# Generate DT algorithm
clf = DecisionTreeClassifier( random_state=0,  max_depth=5)

In [19]:
#  define X,y
X = encoded_pf_2.iloc[:,3:]
y= encoded_pf_2['outcome']

In [20]:
# Data Prepare-> 70:30,(train/test, X,y)

X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.3 )
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((7000, 68), (3000, 68), (7000,), (3000,))

In [21]:
# train
clf.fit( X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=5, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=0, splitter='best')

#### Visualization 
- it's just for showing the structure of clf (the fitted decision tree)
- if you dont' need to check this, you can skip below 2 codes. 

In [44]:
# export the dump file of DT model

export_graphviz(clf, out_file = 'tree.model', 
                class_names   = str(y.unique),
                feature_names = X.columns, 
                filled        = True)

In [45]:


# Load the model dump file
with open('tree.model') as f:
  model_graph = f.read()

# the visualization of DT
graphviz.Source( model_graph )

ExecutableNotFound: failed to execute ['dot', '-Tsvg'], make sure the Graphviz executables are on your systems' PATH

In [46]:
# predict
y_pred = clf.predict( X_test )
y_pred

ValueError: Number of features of the model must match the input. Model n_features is 68 and input n_features is 88 

In [63]:
# check accuracy_score
accuracy_score( y_test, y_pred )

0.6736575933363081

### prefix_Length 5

In [47]:
# Generate DT algorithm
clf1 = DecisionTreeClassifier( random_state=0,  max_depth=5)

In [48]:
#  define X,y
x = encoded_pf_5.iloc[:,3:]
y= encoded_pf_5['outcome']

In [49]:
# Data Prepare-> 70:30,(train/test, X,y)

X_train, X_test, y_train, y_test = train_test_split( x, y, test_size=0.3 )
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((15686, 88), (6723, 88), (15686,), (6723,))

In [50]:
# train
clf1.fit( X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=5, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=0, splitter='best')

In [51]:
# predict
y_pred = clf1.predict( X_test )
y_pred

array([ True,  True,  True, ...,  True,  True,  True])

In [52]:
# check accuracy_score
accuracy_score( y_test, y_pred )

0.6834746392979325

## Random Forest

In [53]:
# 랜덤포레스트 사용 -> 앙상블 기법중 배깅을 이용하여서 예측했다
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier( random_state=0, n_estimators=100, oob_score=True)

### prefix_Length 2


In [54]:
#  define X,y
x = encoded_pf_2.iloc[:,3:]
y= encoded_pf_2['outcome']

In [55]:
# Data Prepare-> 70:30,(train/test, X,y)

X_train, X_test, y_train, y_test = train_test_split( x, y, test_size=0.3 )
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((7000, 68), (3000, 68), (7000,), (3000,))

In [56]:
# RF 훈련
rf_clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=True, random_state=0, verbose=0,
                       warm_start=False)

In [57]:
# RF 예측 및 정확도 체크
pred = rf_clf.predict( X_test )
accuracy_score( y_test, pred )

0.6726666666666666

### prefix_Length 5


In [58]:
rf_clf1 = RandomForestClassifier( random_state=0, n_estimators=100, oob_score=True)

In [59]:
#  define X,y

x = encoded_pf_5.iloc[:,3:]
y= encoded_pf_5['outcome']

In [60]:
# Data Prepare-> 70:30,(train/test, X,y)

X_train, X_test, y_train, y_test = train_test_split( x, y, test_size=0.3 )
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((15686, 88), (6723, 88), (15686,), (6723,))

In [61]:
# RF 훈련

rf_clf1.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=True, random_state=0, verbose=0,
                       warm_start=False)

In [62]:
# RF 예측 및 정확도 체크

pred = rf_clf1.predict( X_test )
accuracy_score( y_test, pred )

0.6812434924884724